In [1]:
#!jupyter nbconvert --to=python Py_f2recom_toolbox.ipynb

In [ ]:
# plot mesh nodal area overview 
def plot_mesh_area(mesh, plot_globe = True, plot_poles=False, plot_zoom=False, levels=[]):
    '''check mesh setup nodal and area
    input: mesh object
    '''
    import pyfesom2 as pf
    import numpy as np
    import matplotlib.cm as cm
    
    meshdiag = pf.get_meshdiag(mesh)
    
    if plot_globe:    
        # whole globe
        pf.plot(mesh,np.array(meshdiag.nod_area[0,:])/1e6,units='nodal area (km$^{2}$)', mapproj='rob',levels=levels, 
        cmap_extension='max', cmap = cm.viridis_r)
        
    if plot_poles:
        # ... and both poles
        pf.plot(mesh,np.array(meshdiag.nod_area[0,:])/1e6,units='nodal area (km$^{2}$)', mapproj='np', box=[-180, 180, 60, 90], cmap = cm.viridis_r,levels=levels)
        pf.plot(mesh,np.array(meshdiag.nod_area[0,:])/1e6,units='nodal area (km$^{2}$)', mapproj='sp', box=[-180, 180, -90, -60], cmap = cm.viridis_r,levels=levels)

    if plot_zoom: 
        # plot nodal area and mesh structure together
        pf.tplot(mesh, np.array(nod_area[0,:])/1e6, ptype='tri', box=[-30, 30, 60, 82], mapproj='np',lw=0.5,units='nodal area')
        pf.tplot(mesh, np.array(nod_area[0,:])/1e6, ptype='tri', box=[-30, 30, -30, 30], mapproj='merc',lw=0.5,units='nodal area')

In [ ]:
# plot mesh resolution overview 
def plot_mesh_resolution(mesh, plot_globe = True, plot_poles=False, plot_zoom=False, levels=[]):
    '''check mesh resolution
    input: mesh objects
    '''
    import pyfesom2 as pf
    import numpy as np
    import matplotlib.cm as cm
    
    meshdiag = pf.get_meshdiag(mesh)
    resolution = np.array(np.sqrt(meshdiag.nod_area[0,:]*np.sqrt(3))/1000)/2
    
    if plot_globe:    
        # whole globe
        pf.plot(mesh,resolution,units='mesh resolution (km)', mapproj='rob',levels=levels, 
        cmap_extension='max', cmap = cm.viridis_r)
        
    if plot_poles:     
        # ... and both poles
        pf.plot(mesh,resolution,units='mesh resolution (km)', mapproj='np', box=[-180, 180, 60, 90], cmap = cm.viridis_r,levels=levels)
        pf.plot(mesh,resolution,units='mesh resolution (km)', mapproj='sp', box=[-180, 180, -90, -60], cmap = cm.viridis_r,levels=levels)

In [ ]:
def load_mat(filename):
    """
    This function should be called instead of direct scipy.io.loadmat
    as it cures the problem of not properly recovering python dictionaries
    from mat files. It calls the function check keys to cure all entries
    which are still mat-objects
    """
    from scipy.io import loadmat, matlab
    import numpy as np
    def _check_vars(d):
        """
        Checks if entries in dictionary are mat-objects. If yes
        todict is called to change them to nested dictionaries
        """
        for key in d:
            if isinstance(d[key], matlab.mio5_params.mat_struct):
                d[key] = _todict(d[key])
            elif isinstance(d[key], np.ndarray):
                d[key] = _toarray(d[key])
        return d

    def _todict(matobj):
        """
        A recursive function which constructs from matobjects nested dictionaries
        """
        d = {}
        for strg in matobj._fieldnames:
            elem = matobj.__dict__[strg]
            if isinstance(elem, matlab.mio5_params.mat_struct):
                d[strg] = _todict(elem)
            elif isinstance(elem, np.ndarray):
                d[strg] = _toarray(elem)
            else:
                d[strg] = elem
        return d

    def _toarray(ndarray):
        """
        A recursive function which constructs ndarray from cellarrays
        (which are loaded as numpy ndarrays), recursing into the elements
        if they contain matobjects.
        """
        if ndarray.dtype != 'float64':
            elem_list = []
            for sub_elem in ndarray:
                if isinstance(sub_elem, matlab.mio5_params.mat_struct):
                    elem_list.append(_todict(sub_elem))
                elif isinstance(sub_elem, np.ndarray):
                    elem_list.append(_toarray(sub_elem))
                else:
                    elem_list.append(sub_elem)
            return np.array(elem_list)
        else:
            return ndarray

    data = loadmat(filename, struct_as_record=False, squeeze_me=True)
    return _check_vars(data)